In [4]:
from dao.attribute import DAOAttribute
from dao.email import DAOEmailGenerated

In [19]:
dao_gen_email: DAOAttribute = DAOEmailGenerated()
pipeline = [
    {
        "$group": {
            "_id": "$og_doc_id",
            "count": {"$sum": 1}
        }
    },
    {
        "$match": {
            "count": {"$gt": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "og_doc_id": "$_id"
        }
    }
]
duplicated_ids = dao_gen_email.collection.aggregate(pipeline)
duplicated_ids_list = [doc['og_doc_id'] for doc in duplicated_ids]

duped_ids = []
if duplicated_ids_list:
    duplicated_documents = dao_gen_email.find_many_by_query({"og_doc_id": {"$in": duplicated_ids_list}})
    for doc in duplicated_documents:
        duped_ids.append([doc.id, doc.og_doc_id])

In [20]:
duped_ids

[[ObjectId('65e65658f8507eaa5a427d90'), ObjectId('65a34a4975da609ba709c829')],
 [ObjectId('65e6568ff8507eaa5a427d91'), ObjectId('65a34a4975da609ba709c829')],
 [ObjectId('65e656acf8507eaa5a427d92'), ObjectId('65a34a4975da609ba709c829')],
 [ObjectId('65e65722f8507eaa5a427d96'), ObjectId('65a3535475da609ba709cacf')],
 [ObjectId('65e65728f8507eaa5a427d98'), ObjectId('65a3564875da609ba709cb97')],
 [ObjectId('65e6575ef8507eaa5a427d99'), ObjectId('65a34a4975da609ba709c829')],
 [ObjectId('65e65777f8507eaa5a427d9d'), ObjectId('65a3564875da609ba709cb97')],
 [ObjectId('65e6577df8507eaa5a427d9e'), ObjectId('65a35a8c75da609ba709cc98')],
 [ObjectId('65e6578bf8507eaa5a427da1'), ObjectId('65a363bf75da609ba709cee7')],
 [ObjectId('65e6578ef8507eaa5a427da2'), ObjectId('65a3657d75da609ba709cf88')],
 [ObjectId('65e657d7f8507eaa5a427daf'), ObjectId('65a36f4975da609ba709d1c2')],
 [ObjectId('65e657e6f8507eaa5a427db3'), ObjectId('65a374b575da609ba709d343')],
 [ObjectId('65e657eaf8507eaa5a427db5'), ObjectId('65

In [24]:
remove_og_ids = []
remove_ids = []
for duped_id in duped_ids:
    if duped_id[1] not in remove_og_ids:
        remove_ids.append(duped_id[0])
        remove_og_ids.append(duped_id[1])

In [25]:
len(remove_ids)

674

In [28]:
dao_attribute = DAOAttribute()
dao_attribute.collection.update_many({"referenced_doc_id": {"$in": [dupe[0] for dupe in duped_ids]}}, {"$set": {"duplicate": True}})

UpdateResult({'n': 1430, 'nModified': 756, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [29]:
dao_attribute.collection.update_many({}, {"$rename": {"duplicated": "duplicate"}})

UpdateResult({'n': 32724, 'nModified': 1430, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [31]:
for doc_id in remove_ids:
    dao_gen_email.update_one({"referenced_doc_id": doc_id}, {"$set": {"duplicate": False}})